In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import proplot as pplt

2021.08.07

将CMORPH和CN05.1资料进行融合，在大陆上使用CN05.1的资料，其他区域用卫星资料补充

In [2]:
dir_in = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/ordata/"
filename_cn051  = "sel_CN05.1_Pre_1961_2018_daily_025x025.nc"
filename_cmorph = "large_CMORPH_98-17.nc"
filename_vr     = "vr_pre_98-17.nc"
filename_rcm    = "rcm_pre_98-17.nc"

ds_or = {}
ds_or['cn051']  = xr.open_dataset(dir_in + filename_cn051)
ds_or['cmorph'] = xr.open_dataset(dir_in + filename_cmorph)
ds_or['vr']     = xr.open_dataset(dir_in + filename_vr)
ds_or['rcm']    = xr.open_dataset(dir_in + filename_rcm  )
# 提取变量
var = {}
var['cn051'] = ds_or['cn051']['pre']
var['cmorph'] = ds_or['cmorph']['cmorph'][:, 0, :, :]
var['vr'] = ds_or['vr']['precip_MPAS']
var['rcm'] = ds_or['rcm']['precip_MPAS']

# # change coords
# var_list = ['cn051','cmorph', 'vr', 'rcm']
# for i in var_list:
#     rename_dict = dict(zip(var[i].coords.keys(), var['cn051'].coords.keys()))
#     # show converting coords
#     for rename_i in rename_dict:
#         print(rename_i + " -----converting to----- " + rename_dict[rename_i])

#     var[i] = var[i].rename(rename_dict)
#     var[i]._coords = var['cn051']._coords
#     var[i] = var[i].rename(i)


/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/coding/times.py:117: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [3]:
# var['cmorph'].time
# var['vr'].Time.dt.month.isin([9]).any()
# var['vr'].Time
lat_small = var['cn051'].lat
lat = var['cmorph'].lat
lon_small = var['cn051'].lon
lon = var['cmorph'].lon
# 获取cmorph中cn051位置的latlon box的 index，是连续的矩形
lat_small_idx = np.searchsorted(lat, lat_small)
lon_small_idx = np.searchsorted(lon, lon_small)

# 合并之后的large domain
merge_cmorph_part = var['cmorph'].copy()

# 需要进行合并的 small domain
cn051_idx = ~np.isnan(var['cn051']) # True有值， False无值
# 有值使用CN05.1没有值使用CMORPH
merge_cn051_part = np.where(cn051_idx, var['cn051'].values, var['cmorph'][:, lat_small_idx, lon_small_idx].values)
# 赋值到large domain
merge_cmorph_part[:, lat_small_idx, lon_small_idx] = merge_cn051_part

In [17]:
# 进行mask之后的检查，确认正确地替换了数据
# mask_res = (merge_cmorph_part[:, lat_small_idx, lon_small_idx] == var['cn051'].values)
# mask_res = (merge_cmorph_part[:, lat_small_idx, lon_small_idx] == var['cmorph'][:, lat_small_idx, lon_small_idx].values)
# plt.contourf(mask_res[0,:,:])

In [14]:
merge_cmorph_part.attrs['long_name'] = 'merge CMORPH & CN05.1'
merge_cmorph_part = merge_cmorph_part.rename('premerge')
# merge_cmorph_part
# merge_cmorph_part = merge_cmorph_part.where(merge_cmorph_part>1e10, -1e30, merge_cmorph_part)


In [15]:
merge_cmorph_part

<xarray.DataArray 'premerge' (time: 3060, lat: 220, lon: 280)>
[188496000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1998-04-01 1998-04-02 ... 2017-08-31
  * lon      (lon) float32 70.12 70.38 70.62 70.88 ... 139.1 139.4 139.6 139.9
  * lat      (lat) float32 5.125 5.375 5.625 5.875 ... 59.12 59.38 59.62 59.88
    lev      float64 1.0
Attributes:
    long_name:  merge CMORPH & CN05.1

## NC文件输出

In [16]:
# 创建临时输出，方便后面再进行计算
dir_out = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA_large/pre/ordata/"
# list(merge_cmorph_part.coords)
# coords_lon  = merge_cmorph_part.lon
# coords_lat  = merge_cmorph_part.lat
# coords_Time = merge_cmorph_part.time
# xr.DataArray(vr_ke_sum,dims = ['longitude', 'latitude', 'Time'])

# temp_write = xr.DataArray(merge_cmorph_part, name = "pre", coords = [coords_Time, coords_lat ,coords_lon],

encoding_dict = {"premerge":{"_FillValue":-1e30}}
merge_cmorph_part.to_netcdf(dir_out + "obsmerge_pre_98-17.nc",'w',\
    encoding = encoding_dict) # 此处进行NAN的处理：修改encoding方法

/m2data2/yycheng/anaconda3/envs/proplot0528/lib/python3.8/site-packages/xarray/coding/times.py:117: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:00. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:00). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
